# Web Scrape Gelest

Let's import the libraries we'll need. Also, hacking the [government pdfs](https://govhack-toolkit.readthedocs.io/technical/scraping/) for more in depth explanation to entity extraction from PDFs.

In [1]:
# import libraries
import PyPDF2
from urllib.request import urlopen
import requests
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import itertools
from bs4 import BeautifulSoup
from rdkit import Chem
from rdkit.Chem import Draw
from util import removeJunk, compoundProperties, getSmiles, getFlashPoint, celsiusToKelvin

In [2]:
text = 'Flash point   :   > 4 °C'
re.search('Flash point(.+?)C', text).group(0)
if '>' not in text or '<' not in text:
    print('good data point')
#re.findall('\d+', fp)
num = ['-1']
int(num[0])

good data point


-1

In [3]:
#helper methods
def getFlashPoint_new(pdfUrl):
    flashp = None
    response = urlopen(pdfUrl)
    file = open("document.pdf", 'wb')
    file.write(response.read())
    file.close()
    pdf_file = open('document.pdf', 'rb')
        
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    
    #number_of_pages = read_pdf.getNumPages()
    for p in range(read_pdf.getNumPages()):
        page = read_pdf.getPage(p)
        text = page.extractText()
        
        text = text.replace('\n', ' ').replace('\r', '')
        fp = re.search('Flash point(.+?)C', text)
        if 'Flash point   :   No' in text or 'Flash point   :   no' in text:
            break
        #print(fp)
        if fp != None:
            fp = re.search('Flash point(.+?)C', text).group(0)
            print(fp)
            if '>' in fp or '<' in fp:
                break
            else:
                flashPoint = re.findall('\d+', fp)
                #print(int(flashPoint[0]))
                flashp = int(flashPoint[0])
                if '-' in fp:
                    flashp = -1 * flashp
                flashp = celsiusToKelvin(flashp)
                print(flashp)
                break
    
    pdf_file.close()
    
    #print(flashp)    
    return flashp


I think we should get the box for each [product](https://www.gelest.com/product-lines/silicones/)

In [4]:
num = 1
frame = pd.DataFrame(columns=("compound", "smiles", "flashpoint", "pure substance", "img", "sds")) #frame for scraped data

while num <= 11:
    print("-------------------Page"+str(num)+"------------------------------------------")
    url = "https://www.gelest.com/product-lines/tin-compounds/?pl_page="+str(num)+"&perpage=20"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    products = soup.find("div", {"class": re.compile("^(first border-box col-xs-24)")})
    contents = soup.find_all("div", {"class": "wc-product-meta"})
    imgs = soup.find_all("img", {"class": "img-responsive center-block wp-post-image"})
    
    for img, data in zip(imgs, contents):
        
        productCode = data.find("span", {"class" : "code"}).text
        sds = soup.find("a", {"href": "https://s3.amazonaws.com/gelest/sds/"+productCode+"_GHS+US_English+US.pdf"})
        casNo = data.find("span", {"class" : "cas-number"})
    
        if casNo == None:
            casNo = ''
        else:
            casNo = casNo.text
        
        compound = data.find("p", {"class" : "title"}).text.strip('\n\t')
        compound = removeJunk(compound)
        #print(compound)
        #smilesUrl = "https://opsin.ch.cam.ac.uk/opsin/"+compound+".smi"
        #print(smilesUrl)
        requestSmiles = " " #requests.get(smilesUrl)
    
        if 1 != 1:#requestSmiles.status_code == 400 or requestSmiles.status_code == 404:
            print("no smiles")
        else:
            try:
                if sds != None: #if sds['href'] != None:
                    #print(sds['href'])
                    if sds == None:
                        flashPoint = None
                    else:
                        flashPoint = getFlashPoint_new(sds['href'])
                        form, binary = compoundProperties(sds['href'])
                        #print(form, binary)
                        frame = frame.append({'compound' : compound,
                            'smiles' : getSmiles(compound), #requestSmiles.text,
                            'flashpoint' : flashPoint,
                            'pure substance' : binary,
                            'img' : img['src'],
                            'sds' : sds['href']}, ignore_index=True)
            except:
                print('could not scrap pdf')


    num+= 1

-------------------Page1------------------------------------------
Flash point   :   > 76 °C
Flash point   :   > 76 °C
Flash point   :   103 °C
376.15
Flash point   :   > 110 °C
Flash point   :   56 °C
329.15
Flash point   :   > 110 °C
Flash point   :   > 110 °C
Flash point   :   > 110 °C
Flash point   :   > 110 °C
Flash point   :   > 110 °C
Flash point   :   168 °C
441.15
Flash point   :   > 110 °C
Flash point   :   > 110 °C
-------------------Page2------------------------------------------
Flash point   :   > 150 °C
Flash point   :   > 150 °C
Flash point   :   > 110 °C
Flash point   :   102 °C
375.15
Flash point   :   > 110 °C
Flash point   :   > 110 °C
Flash point   :   > 110 °C
Flash point   :   39 °C
312.15
Flash point   :   > 110 °C
Flash point   :   > 150 °C
Flash point   :   > 110 °C
Flash point   :   > 110 °C
Flash point   :   101 °C
374.15
Flash point   :   > 204 °C
Flash point   :   > 110 °C
Flash point   :   30 °C
303.15
-------------------Page3-----------------------------

In [5]:
frame.shape

(160, 6)

In [6]:
frame.drop(columns=['sds', 'img'], inplace=True)
frame.dropna(inplace=True)
Chem.MolToSmiles(Chem.MolFromSmiles('C[Ge](Cl)(Cl)Cl'))
#print(frame.shape)
#frame.dropna(inplace=True)
print(frame.shape)


(44, 4)


In [7]:
frame['source'] = 'gelest_tin'
frame.head()


,compound,smiles,flashpoint,pure substance,source
2,ALLYLTRI-n-BUTYLTIN,C=CC[Sn](CCCC)(CCCC)CCCC,376.15,1,gelest_tin
4,BIS[BIS(TRIMETHYLSILYL)AMINO]TIN(II),C[Si](C)(C)N([Sn]N([Si](C)(C)C)[Si](C)(C)C)[Si...,329.15,1,gelest_tin
16,t-BUTYLTRI n-BUTYLTIN,CCCC[Sn](CCCC)(CCCC)C(C)(C)C,375.15,1,gelest_tin
20,CHLOROMETHYLTRIMETHYLTIN,C[Sn](C)(C)CCl,312.15,1,gelest_tin
25,DIALLYLDI-n-BUTYLTIN,C=CC[Sn](CC=C)(CCCC)CCCC,374.15,1,gelest_tin


In [8]:
#frame.dropna(inplace=True) # = frame[frame.flashPoint != None]
print(frame.shape)
frame.to_csv('tin_all.csv', encoding='utf-8',index=False)
#sdsUrl = "https://s3.amazonaws.com/gelest/sds/SIA0482.0_GHS+US_English+US.pdf"
#fp = getFlashPoint(sdsUrl)
#print(fp)

(44, 5)


In [ ]:
intermediates = pd.read_csv('intermediates_all.csv')
metalOrganics = pd.read_csv('metal-organics_all.csv')
tinCompounds = pd.read_csv('tin-compounds_all.csv')
silanes = pd.read_csv('silanes_all.csv')
germaniumCompounds = pd.read_csv('germanium-compounds_all.csv')
silicones = pd.read_csv('silicones_all.csv')

#compounds = silanes.append([intermediates,
#                            tinCompounds,
##                            germaniumCompounds,
#                            silicones,
#                            metalOrganics])

In [ ]:
#find smiles strings
#silanes['smiles'] = silanes['compound'].apply(getSmiles)
#silanes.to_csv("silanes_all.csv",encoding='utf-8',index=False)

In [ ]:
#silanes
#silanes['Product form'] = silanes['sds'].apply(compoundProperties)
silanes

In [ ]:
silanes.iloc[0]
silanes = silanes[pd.notnull(silanes['smiles'])]
silanes = silanes.loc[silanes['Product form'] == 'Substance']
silanes['smiles'].isnull().sum().sum()
silanes.to_csv("gelest_silanes_substance.csv",encoding='utf-8',index=False)
#metalOrganics['smiles'] = metalOrganics['compound'].apply(getSmiles)
#tinCompounds['smiles'] = tinCompounds['compound'].apply(getSmiles)
#germaniumCompounds['smiles'] = germaniumCompounds['compound'].apply(getSmiles)

In [ ]:
#metal organics
metalOrganics['smiles'] = metalOrganics['compound'].apply(getSmiles)
metalOrganics = metalOrganics[pd.notnull(metalOrganics['smiles'])]
print(metalOrganics['smiles'].isnull().sum().sum())
metalOrganics = metalOrganics[pd.notnull(metalOrganics['smiles'])]
metalOrganics['Product form'] = metalOrganics['sds'].apply(compoundProperties)
print(metalOrganics['smiles'].isnull().sum().sum())

In [ ]:
metalOrganics = metalOrganics.loc[metalOrganics['Product form'] == 'Substance']
metalOrganics.to_csv("gelest_metal-organics_substance.csv",encoding='utf-8',index=False)

In [ ]:
# tin compounds
tinCompounds['smiles'] = tinCompounds['compound'].apply(getSmiles)
tinCompounds = tinCompounds[pd.notnull(tinCompounds['smiles'])]
print(tinCompounds['smiles'].isnull().sum().sum())
tinCompounds = tinCompounds[pd.notnull(tinCompounds['smiles'])]
tinCompounds['Product form'] = tinCompounds['sds'].apply(compoundProperties)
print(tinCompounds['smiles'].isnull().sum().sum())

tinCompounds = tinCompounds.loc[tinCompounds['Product form'] == 'Substance']
tinCompounds.to_csv("gelest_tin-compounds_substance.csv",encoding='utf-8',index=False)

In [ ]:
# germanium compounds
germaniumCompounds['smiles'] = germaniumCompounds['compound'].apply(getSmiles)
germaniumCompounds = germaniumCompounds[pd.notnull(germaniumCompounds['smiles'])]
print(germaniumCompounds['smiles'].isnull().sum().sum())
germaniumCompounds = germaniumCompounds[pd.notnull(germaniumCompounds['smiles'])]
germaniumCompounds['Product form'] = germaniumCompounds['sds'].apply(compoundProperties)
print(germaniumCompounds['smiles'].isnull().sum().sum())

germaniumCompounds = germaniumCompounds.loc[germaniumCompounds['Product form'] == 'Substance']
germaniumCompounds.to_csv("gelest_germanium-compounds_substance.csv",encoding='utf-8',index=False)

# Using Opsin web services to request the smiles from IUPAC name

In [ ]:
url = "https://opsin.ch.cam.ac.uk/opsin/ACETOXYETHYLTRIETHOXYSILANE.smi"
r = requests.get(url)
r.text

In [ ]:
m = Chem.MolFromSmiles(r.text)
Draw.MolToImage(m)

m1 = Chem.MolFromSmiles('C(#N)CCC[Si](O[*:2])(C)[*:1]')
Draw.MolToImage(m1)

# Extract identifiers

In [ ]:
response = urlopen('https://s3.amazonaws.com/gelest/sds/SIA0006.0_GHS+US_English+US.pdf')
file = open("document.pdf", 'wb')
file.write(response.read())
file.close()
pdf_file = open('document.pdf', 'rb')
read_pdf = PyPDF2.PdfFileReader(pdf_file)

In [ ]:
page = read_pdf.getPage(0)
text = page.extractText()
text = text.replace('\n', ' ').replace('\r', '')
text

In [ ]:
fp = re.search('Product form(.+?) P', text)
string = fp.group(0)
strArray = string.split()
print(strArray[3])
fp = re.search('Physical state(.+?) F', text)
string = fp.group(0)
strArray = string.split()
print(strArray[3])
fp = re.search('Chemical family(.+?) 1', text)
string = fp.group(0)
strArray = string.split()
print(strArray[3])